# Initalize libraries

## Import libraries

In [ ]:
import os
import sys
import time
from getpass import getuser
from importlib import reload
from os import path
from os.path import join

import h5py
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

# pyFAI
import pyFAI
import xarray as xr
from matplotlib.colors import LogNorm

# Open nexus files
from nexusformat.nexus import *
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector
from scipy.optimize import curve_fit
from tqdm.auto import tqdm

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
# Gifs
import imageio
import interactive
import support_functions as sup
from interactive import cimshow, cimshow_title

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    asd
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
# %load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 20231923
USER = getuser()

### Loading
Take care when loading SOLEIL images and data, as the naming scheme of the data records changes. While the images were sometimes stored under "data_20", they were sometimes stored under "data_18"....

In [ ]:
def load_data(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s%04d.nxs" % (sample_name, scan_id))
    entry = "scan_%04d" % scan_id
    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["scan_data"][key][()])
        print(
            "Loaded key %s of %s; description: %s"
            % (key, entry, f[entry]["scan_data"][key].attrs["description"])
        )
    return np.squeeze(data)


def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False


# Load image files
def load_images(im_ids):
    """
    Load ccd images from nxs files
    """
    key = "data_20"
    images = []
    if is_iterable(im_ids):
        for im_id in im_ids:
            print(im_id)
            images.append(load_data(im_id, key))
    else:
        images.append(load_data(im_ids, key))
    images = np.array(images)
    return images


# Full image loading procedure
def load_processing(im_id, crop=None, binning=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    # images = sup.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    if binning > 1:
        images = np.array([sup.binning(im, binning) for im in images])
        image = sup.binning(image, binning)

    return image, images

In [ ]:
BASEFOLDER = r"C:\Users\gaebel\Desktop\MBI Orga\Beamtimes\24_07_SOLEIL\SAXS_Analysis"
sample_name = "scanx_"

### Masking

In [ ]:
from matplotlib.path import Path
from matplotlib.widgets import PolygonSelector


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [[yc_1,xc_1],[yc_2,xc_2],...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[[yc_1,xc_1],[yc_2,xc_2],...],[[yc_1,xc_1],[yc_2,xc_2],...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])

    # Loop over coordinates
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(polygon_name_list, shape):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    polygon_name_list : list
        keys of different mask coordinates to load

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []

    # Load dictionary of coordinates
    mask_coordinates = load_poly_coordinates()

    # Loop over relevant mask keys
    for polygon_name in polygon_name_list:
        coord = mask_coordinates[polygon_name]
        mask.append(create_polygon_mask(shape, coord).astype(float))

    # Combine all individual mask layers
    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.20,  # ccd to sample distance
    "px_size": 10e-6,  # pixel_size of camera
    "binning": 4,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder
folder_general = sup.create_folder(join(BASEFOLDER, "processed"))
print("Output Folder: %s" % folder_general)

# Load images


In [ ]:
# Specify image ids in iterable like list or array
im_ids = [1715, 1716] #np.hstack(    (np.arange(1574, 1613 + 1), np.arange(1615, 1688+1)))  # np.arange(832, 1013 + 1)  #
dark_id = None #np.ones(len(im_ids)) * 1729


# Which data from nexus files to load? (e.g. "energy","srotz", ...)
key = "energy"  # data_16 is energy

## Normalize?
normalization = False
norm_key = "maxibeckhoff1adc2_rmean"

# Load data
images, scan_axis, norm = [], [], []
for i, im_id in enumerate(tqdm(im_ids)):
    # load images
    _, timages = load_processing(im_id, binning=experimental_setup["binning"])

    ## Load and subtract dark images
    if dark_id is not None:
        dark, _ = load_processing(dark_id[i], binning=experimental_setup["binning"])
        timages = timages - dark

    # Append to data variable
    images.append(timages)

    # Load scan axis
    if key == "energy":
        local_key = "data_16"
    else:
        local_key = key
    try:
        scan_axis.append(load_data(im_id, local_key))
    except:
        scan_axis.append(load_pre_scan_snapshot(im_id, key))  # not implemented yet

    # Load normalization
    if normalization is True:
        norm.append(load_data(im_id, norm_key))
    else:
        norm.append(np.ones(timages.shape[0]))

# Get one lambda value
if key == "energy":
    energy_lambda = cci.photon_energy_wavelength(scan_axis[0])
else:
    energy_lambda = cci.photon_energy_wavelength(
        load_pre_scan_snapshot(im_id, "energy")
    )  # not implemented yet

# Make it beautiful
try:
    scan_axis = np.concatenate(scan_axis)
except:
    scan_axis = np.stack(scan_axis)

try:
    norm = np.concatenate(norm)
except:
    norm = np.stack(norm)


try:
    images = np.concatenate(images)
except:
    images = np.stack(images)


# Squeeze list
images = np.squeeze(images)
scan_axis = np.squeeze(scan_axis)
norm = np.squeeze(norm)

# Mask broken line
# images[:,56]  = 0

# Plot scan axis
fig, ax = plt.subplots()
ax.plot(scan_axis, "o-")
ax.set_title("Scan Axis")
ax.set_ylabel(key)
ax.set_xlabel("Frame Index")
ax.grid()

In [ ]:
# Assign to xarray
# Setup xarray for images
data = xr.Dataset()
data[key] = xr.DataArray(scan_axis, dims=["index"])
# data["norm"] = xr.DataArray(norm, dims=["index"])
data["images"] = xr.DataArray(images, dims=["index", "y", "x"])
data["im_num"] = xr.DataArray(im_ids, dims=["index"])

data["image"] = data["images"].mean("index")
image = data["image"].values

# Sort ascending key
data = data.sortby(key)

# Assign im_id as attribute
data = data.assign_attrs({"im_id": im_id})

# Slideshow viewer
fig, ax = cimshow_title(data["images"], title=data["im_num"].values)

# Draw beamstop mask

In [ ]:
poly_mask = interactive.draw_polygon_mask(image)

In [ ]:
# Take poly coordinates and mask from widget#
p_coord = poly_mask.get_vertice_coordinates()
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
tmp = image * (1 - mask_draw)
mi, ma = np.percentile(tmp[tmp != 0], [0.1, 99.9])
ax[0].imshow(image * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(image * mask_draw, [0.1, 99.9])
ax[1].imshow(image * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["bs_large"] = [
        [
            (792.5189254439257, -36.64236138451321),
            (794.5752135022731, 821.3175414241609),
            (758.8860107217727, 909.1678867121825),
            (772.4134774378779, 1300.3029826015336),
            (745.1593942677341, 1633.9331935948294),
            (668.7711519769113, 2080.9151144836446),
            (1293.9338978126652, 2066.0511051420954),
            (1209.1190304142394, 1784.9259745892543),
            (1214.609676995855, 1230.3706698460942),
            (1221.7338724219853, 723.4140555665841),
            (1223.4139650679383, -29.621668274549677),
        ]
    ]
    mask_coordinates["bs_half"] = [
        [
            (3207.7879576453693, 3079.8664782217993),
            (6110.624356305808, 3154.147691078677),
            (5414.660102079155, 3952.733805050458),
            (5538.179065387015, 4504.451841158061),
            (5719.340211571877, 4726.785975112208),
            (5730.676246301539, 5529.194270553514),
            (5631.896449575138, 5858.795299867034),
            (5749.809977378066, 6157.754096870782),
            (6154.8018785349395, 6155.698754077697),
            (6150.845648024588, -11.12196270770994),
            (-12.265432688581825, -23.02054019250909),
            (-28.27792235696569, 3107.5673387025336),
        ]
    ]
    mask_coordinates["small_patch"] = [
        [
            (2968.984628583505, 5757.757785822321),
            (1355.0035080274533, 5733.053784944657),
            (1338.5343129197386, 4901.359418991554),
            (1643.2144224124627, 4547.271723311507),
            (1585.572239535461, 3822.627138568879),
            (2483.1433729059177, 3797.9233459070942),
            (2540.7855557829203, 4868.421027908557),
            (2985.45382369122, 5140.162747185852),
        ]
    ]
    mask_coordinates["bs_half_binned"] = [
        [
            (-4.45056608472531, 777.0657115369237),
            (798.9507335927591, 772.2692858672076),
            (1527.6126169508343, 785.1509686615527),
            (1373.6137770800633, 971.0055217581673),
            (1361.6227129057727, 1059.2597540809475),
            (1434.5283830854605, 1174.373970154139),
            (1432.1301702506025, 1389.25384015743),
            (1404.7905439332194, 1460.2409400692316),
            (1432.609812817574, 1548.9748149589834),
            (1541.009032953163, 1541.780176454409),
            (1538.1311775513332, -6.026387163048128),
            (-4.878960396408644, -4.587459462133069),
        ]
    ]
    mask_coordinates["small_patch_binned"] = [
        [
            (620.1417260866863, 947.7137954399271),
            (640.7192973080955, 1435.4022192581388),
            (315.5936720098292, 1435.402219258085),
            (315.5936720098292, 1231.6842641661335),
            (379.3841427961979, 1188.4713646011742),
            (414.36601387259367, 1067.0636943948596),
            (402.01947113974813, 947.7137813106859),
        ]
    ]
    mask_coordinates["blob_binned"] = [
        [
            (717.0944677873816, 1461.9950044694635),
            (708.9069272991306, 1469.0660602709538),
            (711.3880001743582, 1476.757386184159),
            (721.5603989627913, 1481.595478290853),
            (732.4771196137924, 1476.6333325403978),
            (735.4544070640655, 1468.1976847646242),
            (726.7706520007691, 1462.2431117569863),
        ]
    ]
    mask_coordinates["weird_thing"] = [
        [
            (496.39199527493946, 878.2475076017331),
            (530.067757794202, 886.4423699638351),
            (530.9640708650568, 909.4904203572466),
            (497.4163530702022, 910.5147781525093),
        ]
    ]
    mask_coordinates["weird_2"] = [
        [
            (395.84619977332534, 1451.2812759259175),
            (764.184724636551, 1447.1657475526101),
            (760.069210392269, 1126.15563649841),
            (393.7884426511844, 1117.9246080098465),
        ]
    ]
    mask_coordinates["weird_3"] = [
        [
            (140.6843166278506, 1120.0166527109877),
            (626.3149974531091, 1120.0166527109877),
            (646.8925686745183, 1264.0596512608527),
            (747.7226676594237, 1278.463951115839),
            (747.7226676594237, 1424.564706787845),
            (546.062469689613, 1422.5069496657038),
            (348.51778596408406, 1369.0052644900397),
            (128.33777389500503, 1202.326937596625),
        ]
    ]
    mask_coordinates["artifacts"] = [
        [
            (821.9567264078662, -19.580862888436968),
            (822.5910206183378, 680.2186485107186),
            (819.1312340157652, 904.2398310272888),
            (814.8065007625496, 988.1396561396723),
            (820.8611273170516, 1093.6631475181339),
            (820.8178799845194, 1460.0977881442723),
            (821.6828266351625, 1592.4346256926708),
            (833.7920797441664, 1585.5150524875257),
            (832.9271330935231, 1461.8276814455585),
            (832.9703804260553, 1095.39304081942),
            (839.0250069805572, 988.1396561396723),
            (834.7002737273415, 906.8346709792181),
            (831.2404871247691, 679.0077152809952),
            (834.9309261675132, -16.986022936507652),
        ],
        [
            (1195.2583534551834, -28.383203160199116),
            (1196.7845802527454, 1058.6366544169787),
            (1198.9927381726225, 1373.7537827116626),
            (1197.6938217491654, 1412.7212754153772),
            (1186.003573938051, 1456.234975601192),
            (1184.7046575145937, 1542.6129177610926),
            (1224.321608430037, 1541.9634595493642),
            (1218.47648452448, 1454.2866009660063),
            (1211.3324441954655, 1412.0718172036486),
            (1213.2808188306512, 1373.104324499934),
            (1213.6704937576883, 1059.286112628707),
            (1214.7420998070406, -21.239162831184785),
        ],
    ]
    mask_coordinates["light"] = [
        [
            (316.3660859177439, 292.4157032683503),
            (315.14012579043475, 356.5742832641945),
            (377.6640922832001, 356.16562988842475),
            (379.2987057862789, 292.00704989258054),
        ],
        [
            (83.51874812860352, 911.7481236421072),
            (79.29541074543837, 1087.0166250434604),
            (189.1021827077319, 1260.1734577532309),
            (348.5331689222158, 1364.701057986568),
            (515.3549955572387, 1430.1627874256276),
            (638.887614014819, 1431.2186217714188),
            (742.3593799023648, 1431.2186217714188),
            (745.5268829397386, 1241.1684395289876),
            (631.49677359428, 1261.2292920990221),
            (628.3292705569061, 1120.8033241087815),
            (202.82802920301862, 1150.3666857909375),
            (192.2696857451058, 904.3572832215682),
        ],
        [
            (373.4943290741294, 703.7166824652222),
            (376.61593692544227, 773.1724571569355),
            (451.5345253569533, 773.1724571569355),
            (446.8521135799838, 748.1995943464319),
            (464.8013587250334, 747.4191923836036),
            (462.46015283654856, 696.6930647997681),
        ],
        [
            (590.5163655381884, 709.7531748404651),
            (590.7478042959906, 745.7419016786992),
            (645.8302286529018, 745.510462920897),
            (643.052963559276, 705.4715578211254),
        ],
        [
            (44.46627813727382, 708.9105114225738),
            (44.466426030297654, 772.2876398807472),
            (109.29052089162583, 773.445213003271),
            (109.29052089162583, 708.042331580681),
        ],
    ]
    mask_coordinates["bs_small_binned"] = [
        [
            (376.9202991522118, -10.20368515072164),
            (425.8917848662426, 82.69810392442494),
            (1555.1218013008472, 80.23321437330057),
            (1546.491042004591, -3.557073794520477),
        ],
        [
            (812.8831010672296, 615.1999684568295),
            (722.6578723021878, 626.0269959086345),
            (699.1993128232768, 653.0945645381471),
            (682.357270120469, 695.801172820267),
            (696.1918051977755, 752.3423161796932),
            (737.6954104296947, 800.4624381877156),
            (787.0185354879177, 811.890967164621),
            (854.9882078242492, 805.8759519136181),
            (880.8527734035613, 772.7933680331028),
            (886.2662871294638, 755.9513253302949),
            (899.4993206816699, 729.4852582258826),
            (861.003223075252, 638.0570264106401),
        ],
        [
            (872.5670388956477, 683.4934697972983),
            (880.6147860424724, 728.3309181867506),
            (1093.3052463514132, 734.6541480878273),
            (1338.0542187606925, 729.4805963505828),
            (1555.918230806878, 723.7322055314223),
            (1548.4453227419692, 681.194113469634),
            (1340.9284141702728, 689.2418606164588),
            (952.4696712819795, 691.5412169441231),
        ],
        [
            (1540.6927074151913, 814.4506291940647),
            (1385.2402835029438, 999.7373566854301),
            (1370.2089747517982, 1070.2152174276239),
            (1404.0830015529134, 1144.394473381549),
            (1444.1237774090982, 1201.7077407835393),
            (1438.627984644524, 1404.9342988970532),
            (1408.7936810654057, 1472.45403857611),
            (1424.495946107047, 1542.289871932711),
            (1540.6927074151913, 1539.463471413142),
            (1538.3373676589451, 945.5645422917682),
        ],
        [
            (722.4111573201803, 1479.643702295281),
            (712.9830803943156, 1486.590706345918),
            (710.2539002315652, 1496.5149978468285),
            (718.1933334322935, 1501.7252508848064),
            (731.8392342460451, 1500.2366071596698),
            (738.041916434114, 1491.800959383896),
            (737.0494872840229, 1485.3501699083045),
        ],
    ]
    mask_coordinates["light_2"] = [
        [
            (402.01947113974813, 944.9701158348946),
            (622.1994832088271, 940.8546013698468),
            (651.0080829188, 1467.6404246379204),
            (257.9764725898833, 1387.3878968744243),
            (97.47141706289113, 1132.2260137289495),
            (408.1927425061709, 1123.9949852403859),
        ]
    ]
    mask_coordinates["light_3"] = [
        [
            (398.2922604909684, 247.5066654416949),
            (398.0770153544949, 310.35824529196015),
            (431.22476637141557, 311.8649612472747),
            (463.7267819789157, 286.68128027987393),
            (464.15727225186276, 247.0761751687479),
        ],
        [
            (323.3185582760583, 681.8372525821446),
            (319.9903553297952, 752.561565190234),
            (392.37876941101604, 753.3936159267997),
            (392.37876941101604, 681.8372525821446),
        ],
        [
            (562.949170406996, 699.3103180500256),
            (562.949170406996, 735.9205504589188),
            (602.0555550255866, 735.0884997223532),
            (599.5594028158894, 700.1423687865913),
        ],
        [
            (427.3249003467778, 898.1704440892413),
            (427.3249003467778, 941.4370823906606),
            (473.9197415944601, 938.1088794443976),
            (473.08769085789436, 899.8345455623728),
        ],
        [
            (590.406844713666, 899.8345455623728),
            (590.406844713666, 937.2768287078319),
            (623.6888741762963, 937.2768287078319),
            (622.0247727031648, 899.8345455623728),
        ],
        [
            (522.6725726833178, 1321.981403792343),
            (526.4584035346919, 1427.9846676308205),
            (756.1321418513928, 1434.2943857164441),
            (751.0843673828939, 1315.6716857067195),
        ],
        [
            (377.5490567139737, 1088.5218346242677),
            (382.59683118247267, 1165.5003952688764),
            (458.31344820995645, 1165.5003952688764),
            (457.0515045928317, 1089.7837782413926),
        ],
        [
            (324.54742479473504, 1335.8627835807151),
            (400.26404182221887, 1380.0308101800806),
            (454.52761735858223, 1332.076952729341),
        ],
        [
            (116.32672796915463, 1150.3570718633796),
            (202.13889393363627, 1247.526730381984),
            (193.30528861376317, 1112.4987633496378),
        ],
        [
            (406.46494632100064, 482.38020059606333),
            (412.2965893623091, 524.8678856113103),
            (455.61736624060006, 526.5340693373984),
            (453.95118251451197, 485.7125680482395),
        ],
        [
            (598.9091666841779, 484.0463843221514),
            (601.40844227331, 527.3671612004424),
            (658.058688960306, 532.3657123787068),
            (653.0601377820416, 489.04493550041576),
        ],
    ]
    mask_coordinates["light_4"] = [
        [
            (420.5594222167224, 684.1072181707273),
            (420.5594222167224, 756.6269985940696),
            (505.7413865235054, 755.4758909683022),
            (501.13695602043606, 678.3516800418906),
        ],
        [
            (83.89096787050396, 903.1185748451663),
            (83.89096787050396, 1103.796019236489),
            (147.61318954926514, 1201.7570465933904),
            (372.06758232818504, 1368.1956853065726),
            (609.8370662041596, 1429.064673178822),
            (723.0153405291234, 1416.7006600172713),
            (711.6024053030766, 1260.723878594632),
            (634.5650925272608, 1268.3325020786633),
            (627.9075469787336, 1125.6708117530784),
            (396.7956086512864, 1137.0837469791254),
            (381.57836168322405, 888.8524058126079),
        ],
        [
            (633.6131598537756, 711.3450303151179),
            (634.1919464150375, 731.8919532399138),
            (652.7131163754169, 730.1555935561282),
            (650.3979701303695, 712.7919967182726),
        ],
        [
            (884.2297210770807, 684.5664721019232),
            (849.0630783489804, 635.4366035799966),
            (813.8964356175094, 614.7503431497116),
            (750.2861847943833, 620.43906476804),
            (718.2224811274417, 643.1939512413535),
            (690.8131860573142, 667.5003072469382),
            (685.1244644389859, 720.2502713441647),
            (705.8107248692708, 754.8997575648918),
            (727.0141418103128, 792.1350263394047),
            (758.0435324557401, 808.6840346836326),
            (816.482218171295, 812.8212867696896),
            (854.2346434565649, 794.7208088931903),
            (879.0581559729068, 764.7257312692773),
            (900.778729424706, 729.5590885377928),
        ],
    ]

    mask_coordinates["light_5"] = [
        [
            (104.61664290885858, 1154.4585844149403),
            (184.26469346394154, 1106.4655283112365),
            (283.31419223116006, 1129.9514919364533),
            (346.62418113391834, 1172.8389037738057),
            (420.14545856937957, 1260.655985155051),
            (428.3144893955419, 1358.6843550689994),
            (403.80739691705486, 1395.4449937867298),
            (236.34226498072655, 1305.585654698944),
        ],
        [
            (493.66673600484074, 1094.0077749987668),
            (491.6244782983002, 1163.4445370211467),
            (565.1457557337613, 1161.4022793146062),
            (562.0823691739505, 1092.9866461454963),
        ],
        [
            (362.96224278624305, 1107.2824500912805),
            (408.91304118340634, 1140.9797022492003),
            (411.97642774321724, 1095.0289038520368),
        ],
        [
            (534.5118901356525, 1426.8957811648827),
            (667.2586410607909, 1441.191585110667),
            (752.0123358822253, 1437.1070696975858),
            (747.9278204691441, 1322.7406381313126),
            (571.2725288533832, 1333.973055517286),
            (524.3006016029495, 1357.4590191425027),
        ],
        [
            (452.3468890429564, 255.05257888300002),
            (454.00495286460443, 326.3493232138672),
            (527.7887929279436, 324.6912593922191),
            (528.6178248387677, 250.90741932887988),
        ],
    ]

    mask_coordinates["bs_bar"] = [
        [
            (717.1154973176726, 677.9072655164432),
            (678.2925111735321, 618.8275185140371),
            (589.203217471208, 621.7634916355929),
            (539.9235224995127, 674.4813048611273),
            (535.9384541623284, 757.242925776817),
            (562.9463385062854, 781.4374888349452),
            (606.834150565203, 805.0693876359076),
            (668.1645545962847, 804.5067233787418),
            (703.807159265848, 795.9614831634457),
            (715.2675534453119, 744.3897093558577),
            (1343.5398402099797, 752.7509894571102),
            (1200.6641465656885, 1034.5988936090646),
            (1251.6999467430717, 1590.2146246244433),
            (1569.6391530690018, 1579.756098100564),
            (1557.0889212403467, 676.1394064373937),
        ]
    ]

    mask_coordinates["light_5"] = [[(381.84499587944555, 690.839730664964), (383.3850541037865, 760.6557035017531), (454.7410851649165, 762.1957617260941), (454.7410851649165, 692.379788889305)], [(592.8329726141537, 703.7676643926133), (593.8596780970477, 738.6756508110078), (647.7617159489805, 740.2157090353487), (647.7617159489805, 702.2276061682724)], [(1096.1623938760672, 659.1691071380535), (1098.6520657150543, 702.3234190138284), (1164.490054346044, 702.0467888094964), (1164.213424141712, 671.3408361286567), (1144.296049429816, 655.296284277407)], [(609.0924505108671, 1117.4442879247017), (567.0063612329225, 1115.8460820027522), (578.726537993887, 1161.6613184319735), (623.4763038084754, 1155.8012300514918), (623.4763038084754, 1116.3788173100686)], [(287.85306019906244, 1317.7438867216333), (372.5579740623905, 1312.416533648468), (378.41806244287227, 1370.4846821459696), (298.507766345393, 1335.8568871703953)], [(175.25922214678107, 854.9956525977228), (174.98649236378247, 889.3596052555431), (213.71412154957994, 887.5868616660523), (212.62320241758565, 854.4501930317256)]]

    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? Use can combine multiple masks, e.g., ["bs_left_part", "bs_bot_part", "bs_top_part"]
polygon_names = ["bs_small_binned", "artifacts"]
mask = load_poly_masks(polygon_names, image.shape)

# optional binning
# mask = sup.binning(mask,experimental_setup["binning"])

# Move beamstop
mask = cci.shift_image(mask, [0, 0])
# mask = mask + cci.circle_mask(mask.shape,[1000,1113],100)

# mask[:, :82] = 1  # broken detector row
# mask[mask > 1] = 1
# mask = np.round(mask)
# mask = mask.astype(int)

# Add to xarray
data["mask"] = xr.DataArray(mask, dims=["y", "x"])

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(image, [0.1, 99.9])
ax[0].imshow(image, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask)")

mi, ma = np.percentile(image * mask, [0.1, 99.9])
ax[1].imshow(image * mask, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask")

ax[2].imshow(1 - mask)
ax[2].set_title("1 - mask")
plt.tight_layout()

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Set center position via widget
c0, c1 = 715, 799  # 3000, 3180 #1026, 1020  # initial values
ic = interactive.InteractiveCenter(data["images"], c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

# Azimuthal integration for all images

In [ ]:
# Update center of azimuthal integrator
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=energy_lambda,
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Do 2d Azimuthal integration of all images and append them to list
list_q, list_i2d = [], []
for i, im in enumerate(tqdm(data["images"].values)):
    # Adapt azimuthal integrator if scan is an energy scan
    if key == "energy":
        ai.wavelength = cci.photon_energy_wavelength(
            data["energy"][i].values, input_unit="eV"
        )

    # Calc ai
    i2d, q, chi = ai.integrate2d(
        im,
        200,
        90,
        radial_range=(0, 1.7),
        unit="q_nm^-1",
        correctSolidAngle=True,
        dummy=np.nan,
        mask=mask,
        method="BBox",
    )
    list_q.append(q)
    list_i2d.append(i2d)

# Add to xarrays
data["q"] = q
data["chi"] = chi
data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])

## Select relevant chi-range

In [ ]:
# Plot 2d and 1d azimuthal integration to estimate the relevant chi and q range
# which image to show?
idx = 1

# Select chi-range
# Which chi-mode? ("all","other")
chi_mode = "all"

# Select chi-range
if chi_mode == "all":
    sel_chi = (data.chi <= 180) * (data.chi >= -180)
elif chi_mode == "other":
    sel_chi = (
        (data.chi <= 100)
        * (data.chi >= -75)
        # + (data.chi <= 47) * (data.chi >= 41)
        # + (data.chi <= -40) * (data.chi >= -48)
        # + (data.chi <= -133) * (data.chi >= -140)
    )
data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")

# Plot
fig, ax = plt.subplots(
    2,
    1,
    figsize=(8, 8),
    sharex=True,
)
mi, ma = np.nanpercentile(data["i2d"][idx], [0.1, 99.9])
data["i2d"][idx].plot.imshow(ax=ax[0], vmin=mi, vmax=ma)
ax[0].set_title(f"2d Azimuthal integration")
ax[0].grid()

# Plot 1d azimuthal integration to estimate the relevant q-range
ax[1].plot(data.q, data.i1d[idx])
# ax[1].set_yscale("log")
ax[1].set_title("1d Azimuthal Integration")
ax[1].grid()
ax[1].set_ylabel("Integrated intensity")
ax[1].set_xlabel("q")

In [ ]:
fig, ax = plt.subplots()
colors = plt.cm.jet(np.linspace(0, 1, len(data[key])))

for i, i1dchi in enumerate(data["i1d"].values):
    # i1dchi = i1dchi - np.nanmin(i1dchi)
    ax.plot(
        data["q"],
        i1dchi / np.nanmax(i1dchi),
        color=colors[i],
        label=data["im_num"][i].values,
    )

ax.grid()
ax.set_xlabel("q")
ax.set_ylabel("Averaged Intensity")#
ax.set_title(f"ScandId: %04d - %04d" % (im_ids[0], im_ids[-1]))
norm = plt.Normalize(np.min(data[key]), np.max(data[key]))
sm = plt.cm.ScalarMappable(cmap="jet", norm=norm)
sm.set_array([])  # You have to set an array for the mappable, can be an empty list
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Energy in eV")
# ax.legend()

In [ ]:
# Plot Intensity(chi) for a given q range
sel = (data.q <= 2) * (data.q >= 0.1)
data["i1dchi"] = data.i2d.where(sel, drop=True).mean("q")

colors = plt.cm.jet(np.linspace(0,1,len(data[key])))

fig, ax = plt.subplots()
for i, i1dchi in enumerate(data["i1dchi"].values):
    ax.plot(data["chi"],i1dchi,color = colors[i])
    
ax.grid()
ax.set_xlabel("chi")
ax.set_ylabel("Averaged Intensity")
ax.set_title(f"ScandId: %04d - %04d"
        % (im_ids[0], im_ids[-1]))
norm = plt.Normalize(np.min(data[key]), np.max(data[key]))
sm = plt.cm.ScalarMappable(cmap='jet', norm=norm)
sm.set_array([])  # You have to set an array for the mappable, can be an empty list
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Energy in eV')

## Select relevant q-range

In [ ]:
# Select relevant q-range for averaging
q0, q1 = 0, 2
binning = False
bins = []

# Get SAXS from q-range
sel = (data.q > q0) * (data.q < q1)

data["saxs"] = data.i1d.where(sel, drop=True).mean("q")

# Averaging of same scan axis values or binning
if binning is True:
    # Execute binning
    data_bin = data.groupby_bins(key, bins).mean()

    # Rename binned values, drop intervals as those cannot be save in h5
    bin_scan_axis = scan_axis + "_bins"
    data_bin = data_bin.swap_dims({bin_scan_axis: key})
    data_bin = data_bin.drop(bin_scan_axis)
else:
    _, count = np.unique(data[key].values, return_counts=True)
    if np.any(count > 1):
        data_bin = data.groupby(key).mean()
    else:
        data_bin = data.swap_dims({"index": key})

# Plotting

In [ ]:
# Plot Intensity of SAXS Pattern
fig, ax = plt.subplots()
ax.plot(data_bin[key].values, data_bin["saxs"].values)
ax.grid()
ax.set_xlabel(key)
ax.set_ylabel("Integrated SAXS")

## Title and fname
if len(im_ids) > 1:
    ax.set_title("Scan Id %s-%s" % (im_ids[0], im_ids[-1]))
    fname = "SAXS_ImId_%04d-%04d_%s.png" % (im_ids[0], im_ids[-1], USER)
else:
    ax.set_title("Scan Id %d" % (im_ids[0]))
    fname = "SAXS_ImId_%04d_%s.png" % (im_ids[0], USER)

fname = join(folder_general, fname)
print("Saving:%s" % fname)
plt.savefig(fname)

In [ ]:
# Plot I(q,t) and integrated intensity
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6, 4.5), sharex=True)

# Create log plot
data_bin["i1dlog"] = np.log10(data_bin["i1d"] + 1)

# vmin, vmax = data_bin["i1d"].min(), data_bin["i1d"].max()
vmin, vmax = np.nanpercentile(data_bin["i1d"], [1, 99])
data_bin["i1d"].plot.contourf(
    x=key,
    y="q",
    ax=ax[0],
    cmap="viridis",
    add_colorbar=False,
    vmin=vmin,
    vmax=vmax,
    levels=100,
    ylim=[0, q1],
)

ax[1].plot(data_bin[key], data_bin["saxs"], "o-")
ax[1].grid()
ax[1].set_ylabel("total scattered intensity")
ax[1].set_xlabel(key)

fig.suptitle(f"ImId:{im_ids}")

fname = join(folder_general, "SAXS_ImId_%04d_%s.png" % (im_ids[0], USER))
print("Saving: %s" % fname)
plt.savefig(fname)

# Export scan as gif

## Select roi of images for plotting

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(data_bin["images"].values)

In [ ]:
# Takes start and end of x and y axis
roi = interactive.axis_to_roi(ax)
print(f"Image registration roi:", roi)

## Plotting

In [ ]:
# Setup gif
folder_gif = sup.create_folder(join(folder_general, "ImId_%05d" % im_id))
variable_images_1d = []

# Find global max and min all images
allmin, allmax = np.nanpercentile(data_bin["i2d"].values, [0.1, 100])
allImin = data_bin.i1d.where(sel, drop=True).min()
allImax = data_bin.i1d.where(sel, drop=True).max()

if allmin < 5:
    allmin = 5

# Loop over images
for i in tqdm(range(len(data_bin[key].values))):
    # Plot for averaged image
    fig = plt.figure(figsize=(6, 10))
    gs1 = gridspec.GridSpec(
        4,
        1,
        figure=fig,
        left=0.2,
        bottom=0.05,
        right=0.975,
        top=1.1,
        wspace=0,
        hspace=0,
        height_ratios=[6, 1, 2, 1],
    )

    # Plot image roi
    ax0 = fig.add_subplot(gs1[0])
    m = ax0.imshow(
        data_bin["images"][i].values[roi] * (1 - mask[roi]), vmin=allmin, vmax=allmax
    )
    plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")

    # Plot 1d azimuthal integration
    ax1 = fig.add_subplot(gs1[1])
    tmp = data_bin.i1d[i]
    ax1.plot(data_bin.q, tmp)
    ax1.set_xlabel("q")
    ax1.set_ylabel("Mean Intensity")
    ax1.set_xlim([0.0, q1])
    ax1.set_ylim([allmin, allImax])
    # ax1.set_yscale("log")
    ax1.grid()
    ax2 = fig.add_subplot(gs1[2])
    vmin, vmax = np.nanpercentile(data_bin["i1d"], [0.1, 99.9])
    data_bin["i1d"].plot.contourf(
        x=key,
        y="q",
        ax=ax2,
        cmap="viridis",
        add_colorbar=False,
        vmin=vmin,
        vmax=vmax,
        levels=200,
        ylim=[0.1, q1],
    )
    ax2.vlines(data_bin[key].values[i], 0.1, q1, "r")
    ax2.hlines(q0, data_bin[key].min(), data_bin[key].max(), "w", linestyles="dashed")
    ax2.hlines(q1, data_bin[key].min(), data_bin[key].max(), "w", linestyles="dashed")

    # Plot SAXS Intensity
    ax3 = fig.add_subplot(gs1[3])
    ax3.plot(data_bin[key].values, data_bin["saxs"].values)
    ax3.scatter(data_bin[key].values[i], data_bin["saxs"].values[i], 20, color="r")
    ax3.set_xlabel(key)
    ax3.set_ylabel("Mean intensity")
    ax3.grid()
    ax3.set_xlim(data_bin[key].min(), data_bin[key].max())

    # Title and fname
    ax0.set_title(
        f"%04d - %04d %s = %s"
        % (im_ids[0], im_ids[-1], key, np.round(data_bin[key].values[i], 3))
    )
    fname = "SAXS_ImId_%04d_%04d_%03d_%s.png" % (im_ids[0], im_ids[-1], i, USER)

    # Save
    fname = path.join(folder_gif, fname)
    variable_images_1d.append(fname)
    plt.savefig(fname)
    plt.close()

# Create gif for 1d AI
if len(im_ids) > 1:
    fname = f"SAXS_ImId_%04d_%04d_%s.gif" % (im_ids[0], im_ids[-1], USER)
else:
    fname = f"SAXS_ImId_%04d_%s.gif" % (im_ids[0], USER)


gif_path = path.join(folder_general, fname)
print("Saving gif:%s" % gif_path)
sup.create_gif(variable_images_1d, gif_path, fps=3)
print("Done!")

In [ ]:
# Drop images
data_bin_save = data_bin.drop_vars(["images"])

# Save log
folder = join(folder_general, "Logs")
sup.create_folder(folder)
fname = join(folder, "SAXS_Log_ImId_%04d_%s.nc" % (im_id, USER))

print("Saving:", fname)
data_bin_save.to_netcdf(fname)

In [ ]:
plt.close("all")

## Fit mulidmodal gaussian

In [ ]:
# Define a combined multimodal Gaussian and exponential decay function
def combined_model(x, *params):
    n_gaussians = (
        len(params) - 3
    ) // 3  # Subtract 3 for the exponential decay parameters (a and b)
    y = np.zeros_like(x)
    for i in range(n_gaussians):
        amplitude = params[3 * i]
        mean = params[3 * i + 1]
        stddev = params[3 * i + 2]
        y += (
            amplitude
            / np.sqrt(np.pi * 2 * stddev**2)
            * np.exp(-((x - mean) ** 2) / (2 * stddev**2))
        )

    # Exponential decay parameters
    a = params[-2]
    b = params[-1]
    c = params[-3]
    d = params[-4]
    # add background
    # y += a*(x+d)**2+b*(x+d)+c
    y += a * np.exp(-b * (x + d)) + c
    # y += a * np.log(b * (x+d)) + c
    return y


# Function to fit data
def fit_combined_model(x_data, y_data, n_gaussians, initial_guess, bounds):
    params, covariance = curve_fit(
        combined_model, x_data, y_data, p0=initial_guess, bounds=bounds
    )
    return params, covariance

### Test multi-gaussian fitting for a selected dataset

In [ ]:
initial_guess = np.array(
    [
        2,
        0.5,
        0.05,
        150,
        0.3,
        0.2,
        -0.29,
        0.2,
        1,
        40,
    ]
)

y_data = data_bin["i1d"].values[0]
x_data = data_bin["q"].values
x_data = x_data[~np.isnan(y_data)][30:-20]
y_data = y_data[~np.isnan(y_data)][30:-20]

bounds = [
    [0, 0, 0, 0, 0, 0, -100, 0, -np.inf, 0],
    [np.inf, 20, np.inf, np.inf, 20, np.inf, np.inf, 100, np.inf, 100],
]
# Fit the data
params, covariance = fit_combined_model(
    x_data, y_data, n_gaussians=2, initial_guess=initial_guess, bounds=bounds
)

# prepare for next round
initial_guess = np.array(params)

# Calculate the standard deviations of the parameters
param_stddevs = np.sqrt(np.diag(covariance))

# Plot the results
plt.figure()
plt.plot(x_data, y_data, label="Data", color="blue")
plt.plot(x_data, combined_model(x_data, *params), label="Fitted Model", color="red")

In [ ]:
initial_guess = np.array(
    [
        2,
        1,
        0.06,
        143,
        0.1,
        0.21,
        -0.29,
        0.19,
        0.92,
        29,
    ]
)

#set index array for cases where the fit did not work properly
failed = []
params_his = [np.zeros(initial_guess.shape)] * len(data_bin["i1d"])
plt.figure()

# loop through 
for i in tqdm(range(len(data_bin["i1d"]))):
    try:
        y_data = data_bin["i1d"].values[i]
        x_data = data_bin["q"].values
        # filter nan values and cut out the interesting region
        x_data = x_data[~np.isnan(y_data)][30:-20]
        y_data = y_data[~np.isnan(y_data)][30:-20]

        bounds = [
            [0, 0, 0, 0, 0, 0, -np.inf, 0, -np.inf, 0],
            [
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
            ],
        ]
        # Fit the data
        params, covariance = fit_combined_model(
            x_data, y_data, n_gaussians=2, initial_guess=initial_guess, bounds=bounds
        )

        # prepare for next round
        # initial_guess = np.mean(np.array([params, start_guess]),axis=0 )

        # Calculate the standard deviations of the parameters
        param_stddevs = np.sqrt(np.diag(covariance))

        # Plot the results
        plt.plot(x_data, y_data, label="Data", color="blue")
        plt.plot(
            x_data, combined_model(x_data, *params), label="Fitted Model", color="red"
        )

    except:
        failed.append(i)
    if np.median(param_stddevs / params) > 1:
        failed.append(i)
    params_his[i] = params
print(len(failed) / len(params_his))


params_his = np.array(params_his)
failed_2 = []
for i in tqdm(failed):
    try:
        y_data = data_bin["i1d"].values[i]
        x_data = data_bin["q"].values
        x_data = x_data[~np.isnan(y_data)][30:-20]
        y_data = y_data[~np.isnan(y_data)][30:-20]
        initial_guess = params_his[i + 3]

        bounds = [
            [0, 0, 0, 0, 0, 0, -np.inf, 0, -np.inf, 0],
            [
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
                np.inf,
            ],
        ]
        # Fit the data
        params, covariance = fit_combined_model(
            x_data, y_data, n_gaussians=2, initial_guess=initial_guess, bounds=bounds
        )

        # prepare for next round
        # initial_guess = np.array(params)

        # Calculate the standard deviations of the parameters
        param_stddevs = np.sqrt(np.diag(covariance))

        # Plot the results
        plt.plot(x_data, y_data, label="Data", color="blue")
        plt.plot(
            x_data, combined_model(x_data, *params), label="Fitted Model", color="red"
        )
        # plt.legend()

    except:
        failed_2.append(i)
    if np.median(param_stddevs / params) > 0.1:
        print(np.median(param_stddevs / params))
    params_his[i] = params
print(len(failed_2) / len(params_his))


plt.xlabel("X")
plt.ylabel("Y")
plt.title("Multimodal Gaussian and Exponential Decay Fit")